# Description
Calculate the proportion of checklists with at least one observation of a given species for each grid cell and year.

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import math
import matplotlib.pyplot as plt
import time

In [2]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/notebooks/calculate_arrival_date'

In [3]:
path = '/Users/alvastrand/Documents/OU/Research/data/'
os.chdir(path)

In [4]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/data'

In [5]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)

In [6]:
start_date = '0101'
end_date = '0731'
month = 'Apr'
year = '2020'
countries_states = 'US_states_east_Mississippi'

In [7]:
subdir = 'output/'
filename = "obligate_aerial_insectivores_ebird_species_codes.csv"

df = pd.read_csv(subdir + filename)

print('len(df) =', len(df))

len(df) = 19


In [8]:
df.head(2)

,scientific_name,common_name,species_code,category,taxon_order,order,family,report_as
0,Antrostomus arizonae,Mexican Whip-poor-will,souwpw1,species,3533,Caprimulgiformes,Caprimulgidae,NaN
1,Antrostomus carolinensis,Chuck-will's-widow,chwwid,species,3510,Caprimulgiformes,Caprimulgidae,NaN


In [9]:
for i in range(len(df)):
      
    print(i, df['species_code'].iloc[i], df['common_name'].iloc[i])

0 souwpw1 Mexican Whip-poor-will
1 chwwid Chuck-will's-widow
2 bucnig Buff-collared Nightjar
3 whip-p1 Eastern Whip-poor-will
4 lesnig Lesser Nighthawk
5 comnig Common Nighthawk
6 compoo Common Poorwill
7 whtswi White-throated Swift
8 chiswi Chimney Swift
9 vauswi Vaux's Swift
10 blkswi Black Swift
11 barswa Barn Swallow
12 cavswa Cave Swallow
13 cliswa Cliff Swallow
14 purmar Purple Martin
15 banswa Bank Swallow
16 nrwswa Northern Rough-winged Swallow
17 treswa Tree Swallow
18 vigswa Violet-green Swallow


In [10]:
def name_of_function(species, start_date, end_date, month, year, random_state, *args):
    
    subdir = 'eBird/ebd_output/'
    
    if args != ():
        
        countries_states = args[0]
        
        filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
        '_complete_zerofilled_grid_cells_rel' + month + '-' + year + '.csv'
        print('filename =', filename)
    
    df = pd.read_csv(subdir + filename)
    print('df.shape =', df.shape)
    
    df = df[['checklist_id', 'observation_date', 'species_observed', 'grid_cell']]
    
    df['species_observed_binary_values'] = np.where(df['species_observed'] == True, 1.0, 0.0)
    df['year'] = df['observation_date'].astype(str).str[:4]
    df['year'] = pd.to_numeric(df['year'])
    
    df = df[['grid_cell', 'year', 'checklist_id', 'observation_date', 'species_observed_binary_values']]
    
    # Filter

    df = df[df['year'] != 2020]
    print('len(df) =', len(df))
    
#     first_year = 2002
#     first_year = 2003
    first_year = 2004

#     first_year = 2010
#     first_year = 2012
#     first_year = 2009
    
#     first_year = 2015
#     first_year = 2014

    end_year = 2019

    start_month = 1
    start_day = 1

    end_month = 6
    # ?
    end_day = 30
    # ?
    
#     for start_year in range(first_year, first_year + 1):
    for start_year in range(first_year, end_year + 1):
        
        start_time = time.time()
        
        print('start_year =', start_year)
        
        # Filter

        df = df[df['year'] >= start_year]
        print('len(df) =', len(df))
        
#         grid_cell = 120
#         grid_cell = 128
#         grid_cell = 136
            
#         grid_cell = 117
#         grid_cell = 147
#         grid_cell = 151

        # Filter
        
#         df = df[df['grid_cell'] == grid_cell]
#         print('len(df) =', len(df))

        df_cnt = df[['checklist_id', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).count()
        df_cnt = df_cnt.rename(columns={'checklist_id': 'nb_checklists'})
        df_cnt = df_cnt.reset_index()
        print('len(df_cnt) =', len(df_cnt))
        print("len(df_cnt['grid_cell'].unique()) =", len(df_cnt['grid_cell'].unique()))
        
        start_date_dt = date(end_year, start_month, start_day)
#         print('start_date_dt =', start_date_dt)

        end_date_dt = date(end_year, end_month, end_day)
#         print('end_date_dt =', end_date_dt)
        
        cnt_days = (end_date_dt - start_date_dt).days + 1
        print('cnt_days =', cnt_days)
        
        julian_days = list(range(1, cnt_days + 1)) * len(range(start_year, end_year + 1))
        assert(len(julian_days) == cnt_days * len(range(start_year, end_year + 1)))
#         print('julian_days[:2] =', julian_days[:2])
        
        dates = [pd.date_range(str(i) + '-01-01', periods=cnt_days) for i in range(start_year, end_year + 1)]
        assert(len(dates) == len(range(start_year, end_year + 1)))
#         print('dates[0][:2] =', dates[0][:2])
        
        dates = [list(date.strftime('%Y-%m-%d')) for date in dates]
        assert(len(dates) == len(range(start_year, end_year + 1)))
#         print('dates[0][:2] =', dates[0][:2])
#         print([date[-1] for date in dates])
        
        dates = [i for sublist in dates for i in sublist]
        assert(len(dates) == cnt_days * len(range(start_year, end_year + 1)))
#         print('dates[:2] =', dates[:2])
        
        df_julian_days_dates = pd.DataFrame({'julian_day': julian_days, 'observation_date': dates})
        df_julian_days_dates = df_julian_days_dates.sort_values(by=['julian_day', 'observation_date'])
        df_julian_days_dates = df_julian_days_dates.reset_index(drop=True)
        assert(len(df_julian_days_dates) == cnt_days * len(range(start_year, end_year + 1)))
        
        df_cnt = df_cnt.merge(df_julian_days_dates, on='observation_date')
        df_cnt = df_cnt[['grid_cell', 'julian_day', 'observation_date', 'nb_checklists']]
        df_cnt = df_cnt.sort_values(by=['grid_cell', 'julian_day', 'observation_date'])
        df_cnt = df_cnt.reset_index(drop=True)
        
        list_grid_cells_repeated = list(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates)
        list_grid_cells_repeated = sorted(list_grid_cells_repeated)
        assert(len(list_grid_cells_repeated) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
#         print('list_grid_cells_repeated[:2] =', list_grid_cells_repeated[:2])
        
        list_julian_days_repeated = list(df_julian_days_dates['julian_day']) * len(df_cnt['grid_cell'].unique())
        assert(len(list_julian_days_repeated) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
#         print('list_julian_days_repeated[:2] =', list_julian_days_repeated[:2])
        
        list_dates_repeated = list(df_julian_days_dates['observation_date']) * len(df_cnt['grid_cell'].unique())
        assert(len(list_dates_repeated) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
#         print('list_dates_repeated[:2] =', list_dates_repeated[:2])
        
        df_grid_cells_days_dates = pd.DataFrame({'grid_cell': list_grid_cells_repeated, 
                                                 'julian_day': list_julian_days_repeated, 
                                                 'observation_date': list_dates_repeated})
        assert(len(df_grid_cells_days_dates) == len(df_cnt['grid_cell'].unique()) * len(df_julian_days_dates))
        
        df_cnt_grid_cells_days_dates = df_grid_cells_days_dates.merge(
            df_cnt, how='left', on=['grid_cell', 'julian_day', 'observation_date'])
        df_cnt_grid_cells_days_dates['nb_checklists'] = df_cnt_grid_cells_days_dates['nb_checklists'].fillna(0)
        df_cnt_grid_cells_days_dates['nb_checklists'] = df_cnt_grid_cells_days_dates['nb_checklists'].astype(int)
        assert(len(df_cnt_grid_cells_days_dates) == len(df_grid_cells_days_dates))
        
        df_min_cnt = df_cnt_grid_cells_days_dates[['grid_cell', 'julian_day', 'nb_checklists']].groupby(
            ['grid_cell', 'julian_day']).min()
        df_min_cnt = df_min_cnt.rename(columns={'nb_checklists': 'min_nb_checklists'})
        df_min_cnt = df_min_cnt.reset_index()
        assert(len(df_min_cnt) == len(df_cnt['grid_cell'].unique()) * cnt_days)
        
        df_cnt_zero = df_min_cnt[['grid_cell', 'min_nb_checklists']].groupby(
            ['grid_cell']).agg(lambda x: x.eq(0).sum())
        df_cnt_zero = df_cnt_zero.rename(columns={'min_nb_checklists': 'cnt_zero'})
        df_cnt_zero = df_cnt_zero.reset_index()
        assert(len(df_cnt_zero) == len(df_cnt['grid_cell'].unique()))
        
        threshold = cnt_days/2
        
        # Filter
        
        list_grid_cells = list(df_cnt_zero[df_cnt_zero['cnt_zero'] < threshold]['grid_cell'])
        print('len(list_grid_cells):', len(list_grid_cells))
        
        print("len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) =", 
              len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]))
        
        
        df_grid_cells_threshold = df[df['grid_cell'].isin(list_grid_cells)]
        print('len(df_grid_cells_threshold):', len(df_grid_cells_threshold))

        # Calculate the number of checklists for each grid cell and date
        df_cnt_not_sampled = df_grid_cells_threshold[['checklist_id', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).count()
        df_cnt_not_sampled = df_cnt_not_sampled.rename(columns={'checklist_id': 'nb_checklists'})
        print(len(df_cnt_not_sampled))

        # Calculate the number of checklists with at least one observation of the species of interest for each grid 
        # cell and date
        df_sum_not_sampled = df_grid_cells_threshold[
            ['species_observed_binary_values', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).sum()
        df_sum_not_sampled = df_sum_not_sampled.rename(
            columns={'species_observed_binary_values': 'nb_checklists_species'})
        assert(len(df_sum_not_sampled) == len(df_cnt_not_sampled))

        df_grp_not_sampled = df_cnt_not_sampled.merge(df_sum_not_sampled, left_index=True, right_index=True)
        # Calculate the proportion of checklists with at least one observation of the species of interest for each 
        # grid cell and date
        df_grp_not_sampled['prop'] = df_grp_not_sampled['nb_checklists_species']/df_grp_not_sampled['nb_checklists']
        df_grp_not_sampled = df_grp_not_sampled.reset_index()
        df_grp_not_sampled['year'] = df_grp_not_sampled['observation_date'].astype(str).str[:4]
        assert(len(df_grp_not_sampled) == len(df_cnt_not_sampled))
        
        df_grid_cells_days_dates_years = df_grid_cells_days_dates[
            df_grid_cells_days_dates['grid_cell'].isin(list_grid_cells)]
        df_grid_cells_days_dates_years = df_grid_cells_days_dates_years.reset_index(drop=True)
        df_grid_cells_days_dates_years['year'] = df_grid_cells_days_dates_years[
            'observation_date'].astype(str).str[:4]
        df_grid_cells_days_dates_years = df_grid_cells_days_dates_years[
            ['grid_cell', 'year', 'julian_day', 'observation_date']]
        assert(len(df_grid_cells_days_dates_years) == 
               len(list_grid_cells) * len(range(start_year, end_year + 1)) * cnt_days)
        
        df_grp_not_sampled_dates = df_grid_cells_days_dates_years.merge(
            df_grp_not_sampled, how='left', on=['grid_cell', 'observation_date', 'year'])
        df_grp_not_sampled_dates = df_grp_not_sampled_dates[
            ['grid_cell', 'year', 'julian_day', 'observation_date', 'nb_checklists', 'nb_checklists_species', 
             'prop']]
        df_grp_not_sampled_dates = df_grp_not_sampled_dates.sort_values(by=['grid_cell', 'year', 'julian_day'])
        df_grp_not_sampled_dates = df_grp_not_sampled_dates.reset_index(drop=True)
        
        # Perform arcsine transformation on the proportions
        df_grp_not_sampled_dates['prop_arcsine'] = df_grp_not_sampled_dates['prop'].apply(
            lambda x: np.arcsin(math.sqrt(x)))
        # ?
        
        assert(len(df_grp_not_sampled_dates) == len(df_grid_cells_days_dates_years))
        
        df_sum_prop_not_sampled = df_grp_not_sampled_dates[['grid_cell', 'year', 'prop']].groupby(
            ['grid_cell', 'year']).sum()
        df_sum_prop_not_sampled = df_sum_prop_not_sampled.reset_index()
        df_sum_prop_not_sampled = df_sum_prop_not_sampled.rename(columns={'prop': 'sum_prop'})
        assert(len(df_sum_prop_not_sampled) == len(list_grid_cells) * len(range(start_year, end_year + 1)))
        
        skip_species = 0
        
        if len(df_sum_prop_not_sampled[df_sum_prop_not_sampled['sum_prop'] == 0]) == len(df_sum_prop_not_sampled):
            
            skip_species = 1
            print('Skip species with species code ' + species + '.')
            
            break
        
        if len(df_sum_prop_not_sampled[df_sum_prop_not_sampled['sum_prop'] == 0]) > 0:
        
            df_sum_prop_zero_not_sampled = df_sum_prop_not_sampled[
                df_sum_prop_not_sampled['sum_prop'] == 0][['grid_cell', 'year']]
            print('len(df_sum_prop_zero_not_sampled) =', len(df_sum_prop_zero_not_sampled))
            print('len(df_sum_prop_not_sampled) - len(df_sum_prop_zero_not_sampled) =', 
                  len(df_sum_prop_not_sampled) - len(df_sum_prop_zero_not_sampled))
            
#             print(df_sum_prop_zero_not_sampled.head(2))
            
            df_grp_not_sampled_dates = df_grp_not_sampled_dates.merge(
                df_sum_prop_zero_not_sampled, how='left', on=['grid_cell', 'year'], indicator=True)
            assert(len(df_grp_not_sampled_dates) == len(df_grid_cells_days_dates_years))
            
            df_grp_not_sampled_dates = df_grp_not_sampled_dates[df_grp_not_sampled_dates['_merge'] == 'left_only']
            print('len(df_grp_not_sampled_dates) =', len(df_grp_not_sampled_dates))
            
            df_grp_not_sampled_dates = df_grp_not_sampled_dates.drop(columns=['_merge'])
                        
        df_sampled = pd.DataFrame()

        for i in range(len(list_grid_cells)):

#             i = 0

            print('i =', i)

            grid_cell_threshold = list_grid_cells[i]

            df_grid_cell = df[df['grid_cell'] == grid_cell_threshold]
            print('len(df_grid_cell) =', len(df_grid_cell))

            df_min_cnt_grid_cell = df_min_cnt[df_min_cnt['grid_cell'] == grid_cell_threshold]
            df_min_cnt_grid_cell = df_min_cnt_grid_cell.reset_index(drop=True)
            assert(len(df_min_cnt_grid_cell) == cnt_days)

            df_min_cnt_greater_than_grid_cell = df_min_cnt_grid_cell[df_min_cnt_grid_cell['min_nb_checklists'] > 0]
            df_min_cnt_greater_than_grid_cell = df_min_cnt_greater_than_grid_cell.reset_index(drop=True)
            print('len(df_min_cnt_greater_than_grid_cell) =', len(df_min_cnt_greater_than_grid_cell))

            # observation_date
            df_min_cnt_greater_than_dates_grid_cell = df_julian_days_dates.merge(
                df_min_cnt_greater_than_grid_cell, on='julian_day')
            assert(len(df_min_cnt_greater_than_dates_grid_cell) == 
                   len(df_min_cnt_greater_than_grid_cell) * len(range(start_year, end_year + 1)))

            list_dates_grid_cell = list(df_min_cnt_greater_than_dates_grid_cell['observation_date'])
            assert(len(list_dates_grid_cell) == len(df_min_cnt_greater_than_dates_grid_cell))

            list_min_cnt_grid_cell = list(df_min_cnt_greater_than_dates_grid_cell['min_nb_checklists'])
            assert(len(list_min_cnt_grid_cell) == len(df_min_cnt_greater_than_dates_grid_cell))
            
            for j in range(len(list_dates_grid_cell)):
                
#                 j = 0

                df_grid_cell_date = df_grid_cell[df_grid_cell['observation_date'] == list_dates_grid_cell[j]]

                df_grid_cell_date = df_grid_cell_date.sample(list_min_cnt_grid_cell[j], random_state=random_state)
#                 assert(len(df_grid_cell_date) == list_min_cnt_grid_cell[j])

                df_sampled = df_sampled.append(df_grid_cell_date)
#                 assert(len(df_sampled) == len(df_grid_cell_date))
            
        print('len(df_sampled):', len(df_sampled))
                
        # Calculate the number of checklists for each grid cell and date
        df_cnt_sampled = df_sampled[['checklist_id', 'grid_cell', 'observation_date']].groupby(
            ['grid_cell', 'observation_date']).count()
        df_cnt_sampled = df_cnt_sampled.rename(columns={'checklist_id': 'nb_checklists'})
        print(len(df_cnt_sampled))

        # Calculate the number of checklists with at least one observation of the species of interest for each grid 
        # cell and date
        df_sum_sampled = df_sampled[['species_observed_binary_values', 'grid_cell', 'observation_date']].groupby([
            'grid_cell', 'observation_date']).sum()
        df_sum_sampled = df_sum_sampled.rename(columns={'species_observed_binary_values': 'nb_checklists_species'})
        assert(len(df_sum_sampled) == len(df_cnt_sampled))

        df_grp_sampled = df_cnt_sampled.merge(df_sum_sampled, left_index=True, right_index=True)
        # Calculate the proportion of checklists with at least one observation of the species of interest for each 
        # grid cell and date
        df_grp_sampled['prop'] = df_grp_sampled['nb_checklists_species']/df_grp_sampled['nb_checklists']
        df_grp_sampled = df_grp_sampled.reset_index()
        df_grp_sampled['year'] = df_grp_sampled['observation_date'].astype(str).str[:4]
        assert(len(df_grp_sampled) == len(df_cnt_sampled))
        
        df_grp_sampled_dates = df_grid_cells_days_dates_years.merge(
            df_grp_sampled, how='left', on=['grid_cell', 'observation_date', 'year'])
        df_grp_sampled_dates = df_grp_sampled_dates[
            ['grid_cell', 'year', 'julian_day', 'observation_date', 'nb_checklists', 'nb_checklists_species', 
             'prop']]
        df_grp_sampled_dates = df_grp_sampled_dates.sort_values(by=['grid_cell', 'year', 'julian_day'])
        df_grp_sampled_dates = df_grp_sampled_dates.reset_index(drop=True)
        
        # Perform arcsine transformation on the proportions
        df_grp_sampled_dates['prop_arcsine'] = df_grp_sampled_dates['prop'].apply(lambda x: np.arcsin(math.sqrt(x)))
        # ?
        
        assert(len(df_grp_sampled_dates) == len(df_grid_cells_days_dates_years))
        
        df_sum_prop_sampled = df_grp_sampled_dates[['grid_cell', 'year', 'prop']].groupby(
            ['grid_cell', 'year']).sum()
        df_sum_prop_sampled = df_sum_prop_sampled.reset_index()
        df_sum_prop_sampled = df_sum_prop_sampled.rename(columns={'prop': 'sum_prop'})
        assert(len(df_sum_prop_sampled) == len(list_grid_cells) * len(range(start_year, end_year + 1)))
        
        # If there are grid cells and years that only have proportions that are equal to zero
        if len(df_sum_prop_sampled[df_sum_prop_sampled['sum_prop'] == 0]) > 0:
        
            df_sum_prop_zero_sampled = df_sum_prop_sampled[
                df_sum_prop_sampled['sum_prop'] == 0][['grid_cell', 'year']]
            print('len(df_sum_prop_zero_sampled) =', len(df_sum_prop_zero_sampled))
            print('len(df_sum_prop_sampled) - len(df_sum_prop_zero_sampled) =', 
                  len(df_sum_prop_sampled) - len(df_sum_prop_zero_sampled))
            
#             print(df_sum_prop_zero_sampled.head(2))
            
            df_grp_sampled_dates = df_grp_sampled_dates.merge(
                df_sum_prop_zero_sampled, how='left', on=['grid_cell', 'year'], indicator=True)
            assert(len(df_grp_sampled_dates) == len(df_grid_cells_days_dates_years))
            
            df_grp_sampled_dates = df_grp_sampled_dates[df_grp_sampled_dates['_merge'] == 'left_only']
            print('len(df_grp_sampled_dates) =', len(df_grp_sampled_dates))
            
            df_grp_sampled_dates = df_grp_sampled_dates.drop(columns=['_merge'])
        
        # If there's at least one grid cell and year that has at least one proportion that's not equal to zero
        if (len(df_sum_prop_sampled[df_sum_prop_sampled['sum_prop'] == 0]) < len(df_sum_prop_sampled)):
    
            if args != ():

                countries_states = args[0]
                
#                 filename_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
#                 end_date + '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_' + \
#                 str(start_year) + '_' + str(end_year) + '_sampled_' + str(random_state) + '_rel' + month + '-' + \
#                 year + '.csv'

                filename_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
                end_date + '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + \
                str(end_year) + '_sampled_' + str(random_state) + '_rel' + month + '-' + year + '.csv'

#                 filename_not_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
#                 end_date + '_complete_zerofilled_grid_cell_' + str(grid_cell) + '_proportions_' + \
#                 str(start_year) + '_' + str(end_year) + '_not_sampled_rel' + month + '-' + year + '.csv'

                filename_not_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + \
                end_date + '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + \
                str(end_year) + '_not_sampled_rel' + month + '-' + year + '.csv'

                print(filename_sampled)
                print(filename_not_sampled)

            df_grp_sampled_dates.to_csv(subdir + filename_sampled, index=False)
            df_grp_not_sampled_dates.to_csv(subdir + filename_not_sampled, index=False)
            
        print('(time.time() - start_time)/60 =', (time.time() - start_time)/60)

    return df, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, df_min_cnt, \
df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, df_grp_not_sampled, \
df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, df_sum_prop_sampled, \
df_sum_prop_not_sampled, skip_species

In [11]:
# species_cnt = 0

# for i in range(len(df)):
    
#     print(i)
  
#     species = df['species_code'].iloc[i]
#     print(species)
    
#     if ((species == 'souwpw1') | (species == 'bucnig') | (species == 'compoo') | (species == 'whtswi') | 
#         (species == 'blkswi') | (species == 'treswa') | (species == 'barswa') | (species == 'chwwid') | 
#         (species == 'whip-p1') | (species == 'lesnig') | (species == 'comnig') | (species == 'chiswi') | 
#         (species == 'vauswi')):
#         continue
    
#     for random_state in range(1, 6):

#         df_grid_cells, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, \
#         df_min_cnt, df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
#         df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
#         df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, \
#         df_grp_not_sampled, df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, \
#         df_sum_prop_sampled, df_sum_prop_not_sampled, skip_species = name_of_function(
#             species, start_date, end_date, month, year, random_state, countries_states)
        
#         if skip_species == 1:
#             break
    
#     species_cnt += 1
    
# print(species_cnt)

In [12]:
# species = 'barswa'
species = 'treswa'

random_state = 1

start_time_all = time.time()

df_grid_cells, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, \
df_min_cnt, df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, \
df_grp_not_sampled, df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, \
df_sum_prop_sampled, df_sum_prop_not_sampled, skip_species = name_of_function(
    species, start_date, end_date, month, year, random_state, countries_states)

# for random_state in range(1, 6):

#     df_grid_cells, df_julian_days_dates, df_cnt, df_grid_cells_days_dates, df_cnt_grid_cells_days_dates, \
#     df_min_cnt, df_cnt_zero, df_grid_cell, df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, \
#     df_min_cnt_greater_than_dates_grid_cell, df_grid_cell_date, df_sampled, df_cnt_sampled, df_sum_sampled, \
#     df_grp_sampled, df_grid_cells_threshold, df_cnt_not_sampled, df_sum_not_sampled, \
#     df_grp_not_sampled, df_grid_cells_days_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, \
#     df_sum_prop_sampled, df_sum_prop_not_sampled, skip_species = name_of_function(
#         species, start_date, end_date, month, year, random_state, countries_states)

print('(time.time() - start_time_all)/60 =', (time.time() - start_time_all)/60)
# ~18 hours

filename = ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_relApr-2020.csv


/Users/alvastrand/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (11,29) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


df.shape = (8351771, 35)
len(df) = 7227884
start_year = 2004
len(df) = 7133501
len(df_cnt) = 221218
len(df_cnt['grid_cell'].unique()) = 108
cnt_days = 181
len(list_grid_cells): 9
len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) = 42
len(df_grid_cells_threshold): 2605335
30250
i = 0
len(df_grid_cell) = 148948
len(df_min_cnt_greater_than_grid_cell) = 173
i = 1
len(df_grid_cell) = 209059
len(df_min_cnt_greater_than_grid_cell) = 139
i = 2
len(df_grid_cell) = 208074
len(df_min_cnt_greater_than_grid_cell) = 119
i = 3
len(df_grid_cell) = 477990
len(df_min_cnt_greater_than_grid_cell) = 166
i = 4
len(df_grid_cell) = 406491
len(df_min_cnt_greater_than_grid_cell) = 178
i = 5
len(df_grid_cell) = 328287
len(df_min_cnt_greater_than_grid_cell) = 161
i = 6
len(df_grid_cell) = 155334
len(df_min_cnt_greater_than_grid_cell) = 111
i = 7
len(df_grid_cell) = 230314
len(df_min_cnt_greater_than_grid_cell) = 180
i = 8
len(df_grid_cell) = 440838
len(df_min_cnt_greater_than_grid_cell) = 170
len(df_sample

i = 30
len(df_grid_cell) = 142049
len(df_min_cnt_greater_than_grid_cell) = 107
len(df_sampled): 278343
59631
len(df_sum_prop_zero_sampled) = 1
len(df_sum_prop_sampled) - len(df_sum_prop_zero_sampled) = 402
len(df_grp_sampled_dates) = 72762
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2007_2019_sampled_1_relApr-2020.csv
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2007_2019_not_sampled_relApr-2020.csv
(time.time() - start_time)/60 = 28.388318697611492
start_year = 2008
len(df) = 7019722
len(df_cnt) = 190802
len(df_cnt['grid_cell'].unique()) = 108
cnt_days = 181
len(list_grid_cells): 38
len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) = 74
len(df_grid_cells_threshold): 5760639
95451
i = 0
len(df_grid_cell) = 77448
len(df_min_cnt_greater_than_grid_cell) = 162
i = 1
len(df_grid_cell) = 93097
len(df_min_cnt_greater_than_grid_cell) = 139
i = 2
len(df_grid_cell) = 187889
len(df_min_cnt_greater_

i = 48
len(df_grid_cell) = 140731
len(df_min_cnt_greater_than_grid_cell) = 177
i = 49
len(df_grid_cell) = 48523
len(df_min_cnt_greater_than_grid_cell) = 147
i = 50
len(df_grid_cell) = 37346
len(df_min_cnt_greater_than_grid_cell) = 92
i = 51
len(df_grid_cell) = 31328
len(df_min_cnt_greater_than_grid_cell) = 93
len(df_sampled): 663047
89749
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2009_2019_sampled_1_relApr-2020.csv
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2009_2019_not_sampled_relApr-2020.csv
(time.time() - start_time)/60 = 60.944815719127654
start_year = 2010
len(df) = 6862236
len(df_cnt) = 167172
len(df_cnt['grid_cell'].unique()) = 108
cnt_days = 181
len(list_grid_cells): 63
len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) = 77
len(df_grid_cells_threshold): 6617901
131772
len(df_sum_prop_zero_not_sampled) = 1
len(df_sum_prop_not_sampled) - len(df_sum_prop_zero_not_sampled) = 629

...

len(df) = 5713346
len(df_cnt) = 105746
len(df_cnt['grid_cell'].unique()) = 104
cnt_days = 181
len(list_grid_cells): 74
len(df_cnt_zero[df_cnt_zero['cnt_zero'] < cnt_days - 1]) = 88
len(df_grid_cells_threshold): 5663912
93553
i = 0
len(df_grid_cell) = 8014
len(df_min_cnt_greater_than_grid_cell) = 107
i = 1
len(df_grid_cell) = 65652
len(df_min_cnt_greater_than_grid_cell) = 181
i = 2
len(df_grid_cell) = 77850
len(df_min_cnt_greater_than_grid_cell) = 181
i = 3
len(df_grid_cell) = 160373
len(df_min_cnt_greater_than_grid_cell) = 181
i = 4
len(df_grid_cell) = 44038
len(df_min_cnt_greater_than_grid_cell) = 180
i = 5
len(df_grid_cell) = 5187
len(df_min_cnt_greater_than_grid_cell) = 95
i = 6
len(df_grid_cell) = 32533
len(df_min_cnt_greater_than_grid_cell) = 181
i = 7
len(df_grid_cell) = 13092
len(df_min_cnt_greater_than_grid_cell) = 170
i = 8
len(df_grid_cell) = 36236
len(df_min_cnt_greater_than_grid_cell) = 181
i = 9
len(df_grid_cell) = 82572
len(df_min_cnt_greater_than_grid_cell) = 181
i = 10


i = 21
len(df_grid_cell) = 11120
len(df_min_cnt_greater_than_grid_cell) = 164
i = 22
len(df_grid_cell) = 22520
len(df_min_cnt_greater_than_grid_cell) = 181
i = 23
len(df_grid_cell) = 49442
len(df_min_cnt_greater_than_grid_cell) = 181
i = 24
len(df_grid_cell) = 41344
len(df_min_cnt_greater_than_grid_cell) = 181
i = 25
len(df_grid_cell) = 62415
len(df_min_cnt_greater_than_grid_cell) = 181
i = 26
len(df_grid_cell) = 66856
len(df_min_cnt_greater_than_grid_cell) = 181
i = 27
len(df_grid_cell) = 51417
len(df_min_cnt_greater_than_grid_cell) = 181
i = 28
len(df_grid_cell) = 46905
len(df_min_cnt_greater_than_grid_cell) = 181
i = 29
len(df_grid_cell) = 22848
len(df_min_cnt_greater_than_grid_cell) = 181
i = 30
len(df_grid_cell) = 23740
len(df_min_cnt_greater_than_grid_cell) = 179
i = 31
len(df_grid_cell) = 29587
len(df_min_cnt_greater_than_grid_cell) = 181
i = 32
len(df_grid_cell) = 19399
len(df_min_cnt_greater_than_grid_cell) = 180
i = 33
len(df_grid_cell) = 14802
len(df_min_cnt_greater_than_gri

i = 44
len(df_grid_cell) = 53502
len(df_min_cnt_greater_than_grid_cell) = 181
i = 45
len(df_grid_cell) = 80945
len(df_min_cnt_greater_than_grid_cell) = 181
i = 46
len(df_grid_cell) = 68628
len(df_min_cnt_greater_than_grid_cell) = 181
i = 47
len(df_grid_cell) = 86156
len(df_min_cnt_greater_than_grid_cell) = 181
i = 48
len(df_grid_cell) = 124409
len(df_min_cnt_greater_than_grid_cell) = 181
i = 49
len(df_grid_cell) = 284875
len(df_min_cnt_greater_than_grid_cell) = 181
i = 50
len(df_grid_cell) = 236632
len(df_min_cnt_greater_than_grid_cell) = 181
i = 51
len(df_grid_cell) = 11560
len(df_min_cnt_greater_than_grid_cell) = 176
i = 52
len(df_grid_cell) = 25920
len(df_min_cnt_greater_than_grid_cell) = 181
i = 53
len(df_grid_cell) = 188275
len(df_min_cnt_greater_than_grid_cell) = 181
i = 54
len(df_grid_cell) = 89547
len(df_min_cnt_greater_than_grid_cell) = 181
i = 55
len(df_grid_cell) = 153331
len(df_min_cnt_greater_than_grid_cell) = 181
i = 56
len(df_grid_cell) = 99581
len(df_min_cnt_greater_tha

i = 61
len(df_grid_cell) = 205796
len(df_min_cnt_greater_than_grid_cell) = 181
i = 62
len(df_grid_cell) = 85420
len(df_min_cnt_greater_than_grid_cell) = 181
i = 63
len(df_grid_cell) = 10435
len(df_min_cnt_greater_than_grid_cell) = 181
i = 64
len(df_grid_cell) = 55346
len(df_min_cnt_greater_than_grid_cell) = 181
i = 65
len(df_grid_cell) = 56772
len(df_min_cnt_greater_than_grid_cell) = 181
i = 66
len(df_grid_cell) = 29201
len(df_min_cnt_greater_than_grid_cell) = 181
i = 67
len(df_grid_cell) = 34200
len(df_min_cnt_greater_than_grid_cell) = 181
i = 68
len(df_grid_cell) = 37598
len(df_min_cnt_greater_than_grid_cell) = 181
i = 69
len(df_grid_cell) = 29754
len(df_min_cnt_greater_than_grid_cell) = 181
i = 70
len(df_grid_cell) = 59804
len(df_min_cnt_greater_than_grid_cell) = 181
i = 71
len(df_grid_cell) = 61058
len(df_min_cnt_greater_than_grid_cell) = 181
i = 72
len(df_grid_cell) = 81629
len(df_min_cnt_greater_than_grid_cell) = 181
i = 73
len(df_grid_cell) = 27966
len(df_min_cnt_greater_than_gr

i = 78
len(df_grid_cell) = 5470
len(df_min_cnt_greater_than_grid_cell) = 179
i = 79
len(df_grid_cell) = 7538
len(df_min_cnt_greater_than_grid_cell) = 180
i = 80
len(df_grid_cell) = 4442
len(df_min_cnt_greater_than_grid_cell) = 104
i = 81
len(df_grid_cell) = 2982
len(df_min_cnt_greater_than_grid_cell) = 119
i = 82
len(df_grid_cell) = 1338
len(df_min_cnt_greater_than_grid_cell) = 130
len(df_sampled): 1924838
29284
len(df_sum_prop_zero_sampled) = 1
len(df_sum_prop_sampled) - len(df_sum_prop_zero_sampled) = 165
len(df_grp_sampled_dates) = 29865
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2018_2019_sampled_1_relApr-2020.csv
ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_proportions_2018_2019_not_sampled_relApr-2020.csv
(time.time() - start_time)/60 = 33.16446274916331
start_year = 2019
len(df) = 1389233
len(df_cnt) = 18213
len(df_cnt['grid_cell'].unique()) = 99
cnt_days = 181
len(list_grid_cells): 85
len(df_cnt_z

In [13]:
df_grid_cells.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
1464,149,2019,S54507126,2019-04-02,0.0
23712,47,2019,S67110419,2019-03-21,1.0


In [14]:
df_julian_days_dates.head(2)

,julian_day,observation_date
0,1,2019-01-01
1,2,2019-01-02


In [15]:
df_cnt.head(2)

,grid_cell,julian_day,observation_date,nb_checklists
0,5,1,2019-01-01,11
1,5,2,2019-01-02,3


In [16]:
df_grid_cells_days_dates.head(2)

,grid_cell,julian_day,observation_date
0,5,1,2019-01-01
1,5,2,2019-01-02


In [17]:
df_cnt_grid_cells_days_dates.head(2)

,grid_cell,julian_day,observation_date,nb_checklists
0,5,1,2019-01-01,11
1,5,2,2019-01-02,3


In [18]:
df_min_cnt.head(2)

,grid_cell,julian_day,min_nb_checklists
0,5,1,11
1,5,2,3


In [19]:
df_cnt_zero.head(2)

,grid_cell,cnt_zero
0,5,4
1,6,53


In [20]:
df_grid_cell.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
131607,170,2019,S51523753,2019-01-10,0.0
132687,170,2019,S52966165,2019-02-17,0.0


In [21]:
df_min_cnt_grid_cell.head(2)

,grid_cell,julian_day,min_nb_checklists
0,170,1,4
1,170,2,1


In [22]:
df_min_cnt_greater_than_grid_cell.head(2)

,grid_cell,julian_day,min_nb_checklists
0,170,1,4
1,170,2,1


In [23]:
df_min_cnt_greater_than_dates_grid_cell.head(2)

,julian_day,observation_date,grid_cell,min_nb_checklists
0,1,2019-01-01,170,4
1,2,2019-01-02,170,1


In [24]:
df_grid_cell_date.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
3478318,170,2019,S57815444,2019-06-30,0.0
8196122,170,2019,G4325928,2019-06-30,0.0


In [25]:
df_sampled.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
1944643,5,2019,S51223809,2019-01-01,0.0
2054687,5,2019,S51675971,2019-01-01,0.0


In [26]:
# df_cnt_sampled.head(2)

In [27]:
# df_sum_sampled.head(2)

In [28]:
df_grp_sampled.head(2)

,grid_cell,observation_date,nb_checklists,nb_checklists_species,prop,year
0,5,2019-01-01,11,0.0,0.0,2019
1,5,2019-01-02,3,0.0,0.0,2019


In [29]:
df_grid_cells_threshold.head(2)

,grid_cell,year,checklist_id,observation_date,species_observed_binary_values
1464,149,2019,S54507126,2019-04-02,0.0
23712,47,2019,S67110419,2019-03-21,1.0


In [30]:
# df_cnt_not_sampled.head(2)

In [31]:
# df_sum_not_sampled.head(2)

In [32]:
df_grp_not_sampled.tail(2)

,grid_cell,observation_date,nb_checklists,nb_checklists_species,prop,year
17681,170,2019-07-30,1,0.0,0.0,2019
17682,170,2019-07-31,1,0.0,0.0,2019


In [33]:
df_grid_cells_days_dates_years.head(2)

,grid_cell,year,julian_day,observation_date
0,5,2019,1,2019-01-01
1,5,2019,2,2019-01-02


In [34]:
df_grp_sampled_dates.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine
0,5,2019,1,2019-01-01,11.0,0.0,0.0,0.0
1,5,2019,2,2019-01-02,3.0,0.0,0.0,0.0


In [35]:
df_grp_not_sampled_dates.head(2)

,grid_cell,year,julian_day,observation_date,nb_checklists,nb_checklists_species,prop,prop_arcsine
0,5,2019,1,2019-01-01,11.0,0.0,0.0,0.0
1,5,2019,2,2019-01-02,3.0,0.0,0.0,0.0


In [36]:
df_sum_prop_sampled.head(2)

,grid_cell,year,sum_prop
0,5,2019,2.750946
1,6,2019,2.509524


In [37]:
df_sum_prop_not_sampled.head(2)

,grid_cell,year,sum_prop
0,5,2019,2.750946
1,6,2019,2.509524


In [ ]:
# df_grp_sampled['prop'].hist()
# plt.show()

In [ ]:
# df_grp_not_sampled['prop'].hist()
# plt.show()